
Generate Preconfig Step by Step Explanation


First, import the necessary Python libraries

In [1]:
import csv
import datetime
import os
from jinja2 import Environment, FileSystemLoader
from pyedgeconnect import Orchestrator

Assign variables

In [2]:
orch_url = input("Please enter FQDN or IP of Orchestrator: ")
print(orch_url)

Please enter FQDN or IP of Orchestrator:  192.168.102.12


192.168.102.12


In [10]:
orch_api_key = input("Please enter Orchestrator API key: ")

Please enter Orchestrator API key:  39c7ef4fc156481fb8866a5305b815164bf841c501dd401886a6ada0b2a9280a131a1757acd947a99b61cf83c7b209a3fda964ee33334ece82fd2e0aebf7107f


In [11]:
#orch_url = '192.168.102.12'
#orch_api_key = '39c7ef4fc156481fb8866a5305b815164bf841c501dd401886a6ada0b2a9280a131a1757acd947a99b61cf83c7b209a3fda964ee33334ece82fd2e0aebf7107f'

Instantiate Orchestrator object

In [12]:
orch = Orchestrator(orch_url, api_key=orch_api_key, verify_ssl=False)

Print orchestrator version as a test to ensure data can be accessed from the Orchestrator object

In [13]:
print(f"Orchestrator version:  {orch.orch_version}")


Orchestrator version:  9.5


Retrieve Jinja2 template for generating EdgeConnect Preconfig YAML
Setting ``trim_blocks`` and ``lstrip_blocks`` reduces excessive
whitepsace from the jinja template conditionals etc.

In [14]:
ec_template_file = 'ec_preconfig_template.jinja2'

env = Environment(
    loader=FileSystemLoader("templates"),
    trim_blocks=True,
    lstrip_blocks=True,
)
ec_template = env.get_template(ec_template_file)

Set output directory for generated preconfig YAML files

In [15]:
# Local directory for configuration outputs
output_directory = "preconfig_outputs/"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

Open CSV file with configuration data and iterate over each row and generate a preconfig per device (row) using the data in the preconfig.csv file with the Jinja template in the templates directory. Each device's preconfig YAML file is validated with the Orchestrator for validity. Regardless of validity, the code below will copy the output of the YAML file to the preconfig_outputs directory for review (if the validation fails, the filename will be suffixed with "_preconfig-FAILED.yml". If the upload_to_orch boolean is set to True, the validated preconfig file will be sent to the orchestrator in the 'Preconfigure Appliances' tab. Preconfig YAML files that fail validation will not be sent to the Orchestrator.

In [18]:
# Open CSV file with configuration data and iterate over each row and generate a preconfig per device (row)
upload_to_orch = True
csv_filename = 'preconfig.csv'
with open(csv_filename, encoding="utf-8-sig") as csvfile:
    csv_dict = csv.DictReader(csvfile)
    # Set initial row number for row identification of data
    # First row is headers
    row_number = 2
    # Generate Edge Connect YAML preconfig for each row in data
    for row in csv_dict:
        # Render CSV values through the Jinja template
        yaml_preconfig = ec_template.render(data=row)
        # Set value for serial number if provided
        appliance_serial = row.get("appliance_serial")
        if appliance_serial is None:
            appliance_serial = ""
        else:
            pass
        # Validate preconfig via Orchestrator
        validate = orch.validate_preconfig(
            preconfig_name=row["hostname"],
            yaml_preconfig=yaml_preconfig,
            auto_apply=True,
        )
        # Write local YAML file to see resulting YAML file locally
        # whether or not validation passes or fails
        yaml_filename = f'{row["hostname"]}_preconfig.yml'
        with open(output_directory + yaml_filename, "w") as preconfig_file:
            write_data = preconfig_file.write(yaml_preconfig)

        # If the validate function passes on Orchestrator, move on
        # to check if uploading to Orchestrator option selected
        if validate.status_code == 200:
            # If upload option was chosen, upload preconfig to
            # Orchestrator with selected auto-apply settings
            if upload_to_orch is True:
                # In this example the appliance hostname from the CSV
                # data (row["hostname"]) is used both for the name of
                # the preconfig to appear in Orchestrator, as well as
                # the tag on the preconfig that could be used to match
                # against a discovered appliance
                # Additionally a comment is added with the current
                # date
                comment_timestamp = datetime.date.today().strftime("%d %B %Y")
                orch.create_preconfig(
                    preconfig_name=row["hostname"],
                    yaml_preconfig=yaml_preconfig,
                    auto_apply=True,
                    tag=row["hostname"],
                    serial_number=appliance_serial,
                    comment=f"Created/Uploaded @ {comment_timestamp}",
                )
                print(f'Posted EC Preconfig {row["hostname"]}')
            else:
                pass
        else:
            print(
                f'Preconfig for {row["hostname"]}'
                f" failed validation | error: {validate.text}"
            )
            # Write local YAML file of failed config for reference
            yaml_filename = f'{row["hostname"]}_preconfig-FAILED.yml'
            with open(output_directory + yaml_filename, "w") as preconfig_file:
                write_data = preconfig_file.write(yaml_preconfig)

        # Increment row number when iterating to next row in CSV
        row_number += 1

[POST] /gms/appliance/preconfiguration/validate | Received HTTP 400 | Response text: Appliance region invalid: US. Valid regions are: [NewMexico, Texas]


Posted EC Preconfig EDGECONNECT1
Posted EC Preconfig EDGECONNECT2
Posted EC Preconfig EDGECONNECT3
Preconfig for WacoEC1 failed validation | error: Appliance region invalid: US. Valid regions are: [NewMexico, Texas]
